# F1 All Time Races Analysis

In [73]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import re

#### Importing all the csv files

In [56]:
df = pd.read_csv('../csv_files/races_info.csv', index_col = [0])
df_drivers = pd.read_csv('../csv_files/drivers_info.csv', index_col = [0])
df_circuits = pd.read_csv('../csv_files/circuits_info.csv', index_col = [0])
df_constructors = pd.read_csv('../csv_files/constructors_info.csv', index_col = [0])

In [57]:
df_constructors = df_constructors.transpose()
df_drivers = df_drivers.transpose()
df_circuits = df_circuits.transpose()

In [58]:
df_constructors = df_constructors[['constructorId', 'name', 'nationality']]
df_drivers = df_drivers[['driverId','givenName','familyName','dateOfBirth', 'nationality']]
df_circuits = df_circuits[['circuitId','circuitName']]

In [59]:
merge_constructors = df.merge( df_constructors, left_on = 'constructor', right_on = 'constructorId')
merge_constructors

,driver,year,round,circuit,constructor,status,position,points,laps,race_time,fastest_lap_time,lap,fastest_lap_rank,average_speed,constructorId,name,nationality
0,alesi,1999,1,albert_park,sauber,Gearbox,21,0,0,NaN,['N/A'],['N/A'],['N/A'],['N/A'],sauber,Sauber,Swiss
1,alesi,1999,2,interlagos,sauber,Gearbox,17,0,27,NaN,['N/A'],['N/A'],['N/A'],['N/A'],sauber,Sauber,Swiss
2,alesi,1999,3,imola,sauber,+1 Lap,6,1,61,NaN,['N/A'],['N/A'],['N/A'],['N/A'],sauber,Sauber,Swiss
3,alesi,1999,4,monaco,sauber,Suspension,11,0,50,NaN,['N/A'],['N/A'],['N/A'],['N/A'],sauber,Sauber,Swiss
4,alesi,1999,5,catalunya,sauber,Transmission,18,0,27,NaN,['N/A'],['N/A'],['N/A'],['N/A'],sauber,Sauber,Swiss
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,irvine,2000,13,spa,jaguar,Finished,10,0,44,+1:31.555,['N/A'],['N/A'],['N/A'],['N/A'],jaguar,Jaguar,British
721,irvine,2000,14,monza,jaguar,Spun off,22,0,0,NaN,['N/A'],['N/A'],['N/A'],['N/A'],jaguar,Jaguar,British
722,irvine,2000,15,indianapolis,jaguar,Finished,7,0,73,+1:11.115,['N/A'],['N/A'],['N/A'],['N/A'],jaguar,Jaguar,British
723,irvine,2000,16,suzuka,jaguar,+1 Lap,8,0,52,NaN,['N/A'],['N/A'],['N/A'],['N/A'],jaguar,Jaguar,British


In [60]:
merge_drivers = merge_constructors.merge( df_drivers, left_on = 'driver', right_on = 'driverId')
merge_drivers

,driver,year,round,circuit,constructor,status,position,points,laps,race_time,...,fastest_lap_rank,average_speed,constructorId,name,nationality_x,driverId,givenName,familyName,dateOfBirth,nationality_y
0,alesi,1999,1,albert_park,sauber,Gearbox,21,0,0,NaN,...,['N/A'],['N/A'],sauber,Sauber,Swiss,alesi,Jean,Alesi,1964-06-11,French
1,alesi,1999,2,interlagos,sauber,Gearbox,17,0,27,NaN,...,['N/A'],['N/A'],sauber,Sauber,Swiss,alesi,Jean,Alesi,1964-06-11,French
2,alesi,1999,3,imola,sauber,+1 Lap,6,1,61,NaN,...,['N/A'],['N/A'],sauber,Sauber,Swiss,alesi,Jean,Alesi,1964-06-11,French
3,alesi,1999,4,monaco,sauber,Suspension,11,0,50,NaN,...,['N/A'],['N/A'],sauber,Sauber,Swiss,alesi,Jean,Alesi,1964-06-11,French
4,alesi,1999,5,catalunya,sauber,Transmission,18,0,27,NaN,...,['N/A'],['N/A'],sauber,Sauber,Swiss,alesi,Jean,Alesi,1964-06-11,French
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,button,2000,14,monza,williams,Spun off,15,0,10,NaN,...,['N/A'],['N/A'],williams,Williams,British,button,Jenson,Button,1980-01-19,British
721,button,2000,15,indianapolis,williams,Engine,21,0,14,NaN,...,['N/A'],['N/A'],williams,Williams,British,button,Jenson,Button,1980-01-19,British
722,button,2000,16,suzuka,williams,Finished,5,2,53,+1:25.694,...,['N/A'],['N/A'],williams,Williams,British,button,Jenson,Button,1980-01-19,British
723,button,2000,17,sepang,williams,Engine,18,0,18,NaN,...,['N/A'],['N/A'],williams,Williams,British,button,Jenson,Button,1980-01-19,British


In [61]:
merge_circuits = merge_drivers.merge( df_circuits, left_on = 'circuit', right_on = 'circuitId')
clean_df = merge_circuits

In [62]:
clean_df['driver_name'] = clean_df['givenName'] + " " + clean_df['familyName']
clean_df = clean_df.drop(['driver','driverId', 'givenName', 'familyName','constructorId', 'circuitId', 'circuit','constructor'], axis = 1)

In [63]:
clean_df = clean_df.rename(columns = {'circuitName':'circuit_name','name':'constructor','nationality_x':'constructor_nationality','nationality_y':'drivers_nationality', 'dateOfBirth':'birth'})
clean_df


,year,round,status,position,points,laps,race_time,fastest_lap_time,lap,fastest_lap_rank,average_speed,constructor,constructor_nationality,birth,drivers_nationality,circuit_name,driver_name
0,1999,1,Gearbox,21,0,0,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Sauber,Swiss,1964-06-11,French,Albert Park Grand Prix Circuit,Jean Alesi
1,2000,1,Hydraulics,16,0,27,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Prost,French,1964-06-11,French,Albert Park Grand Prix Circuit,Jean Alesi
2,1999,1,Transmission,12,0,27,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Sauber,Swiss,1970-05-22,Brazilian,Albert Park Grand Prix Circuit,Pedro Diniz
3,2000,1,Transmission,12,0,41,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Sauber,Swiss,1970-05-22,Brazilian,Albert Park Grand Prix Circuit,Pedro Diniz
4,2000,1,Disqualified,10,0,58,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Sauber,Swiss,1966-11-30,Finnish,Albert Park Grand Prix Circuit,Mika Salo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,2000,15,+1 Lap,9,0,72,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Prost,French,1977-05-10,German,Indianapolis Motor Speedway,Nick Heidfeld
721,2000,15,Finished,4,3,73,+17.936,['N/A'],['N/A'],['N/A'],['N/A'],BAR,British,1971-04-09,Canadian,Indianapolis Motor Speedway,Jacques Villeneuve
722,2000,15,Finished,6,1,73,+51.694,['N/A'],['N/A'],['N/A'],['N/A'],BAR,British,1976-03-23,Brazilian,Indianapolis Motor Speedway,Ricardo Zonta
723,2000,15,Engine,15,0,58,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Williams,British,1975-06-30,German,Indianapolis Motor Speedway,Ralf Schumacher


In [64]:
df = clean_df[['year','driver_name','constructor', 'circuit_name', 'round','status', 'position', 'points', 'laps', 'race_time', 'fastest_lap_time','lap','fastest_lap_rank','average_speed', 'drivers_nationality','birth', 'constructor_nationality']]
df

,year,driver_name,constructor,circuit_name,round,status,position,points,laps,race_time,fastest_lap_time,lap,fastest_lap_rank,average_speed,drivers_nationality,birth,constructor_nationality
0,1999,Jean Alesi,Sauber,Albert Park Grand Prix Circuit,1,Gearbox,21,0,0,NaN,['N/A'],['N/A'],['N/A'],['N/A'],French,1964-06-11,Swiss
1,2000,Jean Alesi,Prost,Albert Park Grand Prix Circuit,1,Hydraulics,16,0,27,NaN,['N/A'],['N/A'],['N/A'],['N/A'],French,1964-06-11,French
2,1999,Pedro Diniz,Sauber,Albert Park Grand Prix Circuit,1,Transmission,12,0,27,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Brazilian,1970-05-22,Swiss
3,2000,Pedro Diniz,Sauber,Albert Park Grand Prix Circuit,1,Transmission,12,0,41,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Brazilian,1970-05-22,Swiss
4,2000,Mika Salo,Sauber,Albert Park Grand Prix Circuit,1,Disqualified,10,0,58,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Finnish,1966-11-30,Swiss
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,2000,Nick Heidfeld,Prost,Indianapolis Motor Speedway,15,+1 Lap,9,0,72,NaN,['N/A'],['N/A'],['N/A'],['N/A'],German,1977-05-10,French
721,2000,Jacques Villeneuve,BAR,Indianapolis Motor Speedway,15,Finished,4,3,73,+17.936,['N/A'],['N/A'],['N/A'],['N/A'],Canadian,1971-04-09,British
722,2000,Ricardo Zonta,BAR,Indianapolis Motor Speedway,15,Finished,6,1,73,+51.694,['N/A'],['N/A'],['N/A'],['N/A'],Brazilian,1976-03-23,British
723,2000,Ralf Schumacher,Williams,Indianapolis Motor Speedway,15,Engine,15,0,58,NaN,['N/A'],['N/A'],['N/A'],['N/A'],German,1975-06-30,British


In [65]:
#number of points per year per constructor, ordered by points
constructor_year_points = df.groupby(['year', 'constructor']).agg({'points':'sum'})
constructor_year_points.sort_values(by = 'points', ascending = False)

points
year constructor        
2000 Ferrari         170
     McLaren         162
1999 Ferrari         128
     McLaren         124
     Jordan           61
2000 Williams         36
1999 Stewart          36
     Williams         35
2000 Benetton         20
     BAR              20
     Jordan           17
1999 Benetton         16
     Prost             9
2000 Arrows            7
     Sauber            6
1999 Sauber            5
2000 Jaguar            4
1999 Minardi           1
     Arrows            1
     BAR               0
2000 Minardi           0
     Prost             0

##### Number of constructors

In [66]:
#number of all time constructors
len(df.circuit_name.unique())

17

#### Top 10 drivers with more point in history

In [67]:
#Top 10 drivers with more points in history
top_points_drivers = df.groupby(['driver_name']).agg({'points':'sum'})
top_points_drivers.sort_values(by='points', ascending = False).head(10)

,points
driver_name,
Mika Häkkinen,165
Michael Schumacher,152
David Coulthard,121
Rubens Barrichello,83
Eddie Irvine,78
Heinz-Harald Frentzen,65
Ralf Schumacher,59
Giancarlo Fisichella,31
Jacques Villeneuve,17


#### Total number of points per season

In [68]:
#total number of points per season
total_season_points = df.groupby(['year']).agg({'points':'sum'}).sort_values(by='year')
total_season_points

,points
year,
1999,416
2000,442


#### Top 10 drivers with most races

In [69]:
top_drivers = df.groupby(['driver_name']).agg({'points':'count', 'position':'median'}).sort_values(by='points', ascending = False)
top_drivers.head(10) 

,points,position
driver_name,,
Pedro de la Rosa,33,16.0
Jacques Villeneuve,33,14.0
Marc Gené,33,14.0
Alexander Wurz,33,10.0
Johnny Herbert,33,12.0
Pedro Diniz,33,12.0
Jean Alesi,33,16.0
Mika Häkkinen,33,2.0
Jarno Trulli,33,13.0


#### Top 10 circuits

In [70]:
top_circuits = df.groupby(['year','circuit_name']).agg({'points':'count'}).sort_values(by='points', ascending = False)
top_circuits = top_circuits.groupby('circuit_name').agg({'points':'count'}).sort_values(by='points', ascending = False)
top_circuits.head(10)

,points
circuit_name,
A1-Ring,2
Albert Park Grand Prix Circuit,2
Silverstone Circuit,2
Sepang International Circuit,2
Nürburgring,2
Hungaroring,2
Hockenheimring,2
Circuit de Spa-Francorchamps,2
Circuit de Nevers Magny-Cours,2


#### Most common disqualifying reasons

In [71]:
#Finding the patter + Lap, to discard the disqualifying reasons + finished
pattern = '\+\d*\s\w*'
text = str(df['status'].unique())
result = re.findall(pattern, text) + ['Finished']
result

['+1 Lap', '+2 Laps', '+3 Laps', '+6 Laps', '+4 Laps', 'Finished']

In [72]:
#creating a dataframe in order to discard the laps
disc_reason = df[~df['status'].isin(result)]
disc_reason

,year,driver_name,constructor,circuit_name,round,status,position,points,laps,race_time,fastest_lap_time,lap,fastest_lap_rank,average_speed,drivers_nationality,birth,constructor_nationality
0,1999,Jean Alesi,Sauber,Albert Park Grand Prix Circuit,1,Gearbox,21,0,0,NaN,['N/A'],['N/A'],['N/A'],['N/A'],French,1964-06-11,Swiss
1,2000,Jean Alesi,Prost,Albert Park Grand Prix Circuit,1,Hydraulics,16,0,27,NaN,['N/A'],['N/A'],['N/A'],['N/A'],French,1964-06-11,French
2,1999,Pedro Diniz,Sauber,Albert Park Grand Prix Circuit,1,Transmission,12,0,27,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Brazilian,1970-05-22,Swiss
3,2000,Pedro Diniz,Sauber,Albert Park Grand Prix Circuit,1,Transmission,12,0,41,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Brazilian,1970-05-22,Swiss
4,2000,Mika Salo,Sauber,Albert Park Grand Prix Circuit,1,Disqualified,10,0,58,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Finnish,1966-11-30,Swiss
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,2000,Jos Verstappen,Arrows,Indianapolis Motor Speedway,15,Brakes,18,0,34,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Dutch,1972-03-04,British
714,2000,Giancarlo Fisichella,Benetton,Indianapolis Motor Speedway,15,Engine,17,0,44,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Italian,1973-01-14,Italian
717,2000,Jarno Trulli,Jordan,Indianapolis Motor Speedway,15,Collision,22,0,12,NaN,['N/A'],['N/A'],['N/A'],['N/A'],Italian,1974-07-13,Irish
723,2000,Ralf Schumacher,Williams,Indianapolis Motor Speedway,15,Engine,15,0,58,NaN,['N/A'],['N/A'],['N/A'],['N/A'],German,1975-06-30,British


In [294]:
top_disq = disc_reason.groupby(['status']).agg({'points':'count'}).sort_values(by='points', ascending = False)
top_disq.head(10)

,points
status,
Engine,1978
Did not qualify,1024
Accident,1015
Collision,791
Gearbox,787
Spun off,785
Suspension,423
Did not prequalify,330
Transmission,318


#### Drivers who have won more races

In [296]:
position_1 = df[df.position == 1]
most_races = position_1.groupby(['driver_name']).agg({'points':'count'}).sort_values(by='points', ascending = False)
most_races.head(10)

,points
driver_name,
Michael Schumacher,91
Lewis Hamilton,84
Sebastian Vettel,53
Alain Prost,51
Ayrton Senna,41
Fernando Alonso,32
Nigel Mansell,31
Jackie Stewart,27
Jim Clark,25


#### Top 10 most 1 positions by nationality

In [299]:
position_1nat = df[df.position == 1]
most_races_nat = position_1nat.groupby(['drivers_nationality']).agg({'points':'count'}).sort_values(by='points', ascending = False)
most_races_nat.head(10)

,points
drivers_nationality,
British,289
German,179
Brazilian,101
French,79
Finnish,54
Italian,43
Australian,42
Austrian,41
Argentine,38


#### Top 10 most 1 positions by constructor

In [302]:
position_1cons = df[df.position == 1]
most_races_cons = position_1cons.groupby(['constructor']).agg({'points':'count'}).sort_values(by='points', ascending = False)
most_races_cons.head(10)

,points
constructor,
Ferrari,239
McLaren,178
Williams,114
Mercedes,102
Red Bull,62
Team Lotus,45
Renault,35
Benetton,27
Brabham,23


In [15]:
df[df.year==2005]

,year,driver_name,constructor,circuit_name,round,status,position,points,laps,race_time,fastest_lap_time,lap,fastest_lap_rank,average_speed,drivers_nationality,birth,constructor_nationality
365,2005,Michael Schumacher,Ferrari,Indianapolis Motor Speedway,9,Finished,1,10.0,73,1:29:43.181,1:11.497,48.0,1.0,211.074,German,1969-01-03,Italian
374,2005,Rubens Barrichello,Ferrari,Indianapolis Motor Speedway,9,Finished,2,8.0,73,+1.5,1:11.649,48.0,2.0,210.626,Brazilian,1972-05-23,Italian
380,2005,Felipe Massa,Sauber,Indianapolis Motor Speedway,9,Withdrew,14,0.0,0,NaN,NaN,NaN,NaN,NaN,Brazilian,1981-04-25,Swiss
385,2005,Kimi Räikkönen,McLaren,Indianapolis Motor Speedway,9,Withdrew,8,0.0,0,NaN,NaN,NaN,NaN,NaN,Finnish,1979-10-17,British
388,2005,Giancarlo Fisichella,Renault,Indianapolis Motor Speedway,9,Withdrew,10,0.0,0,NaN,NaN,NaN,NaN,NaN,Italian,1973-01-14,French
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23226,2005,Robert Doornbos,Minardi,Istanbul Park,14,+3 Laps,13,0.0,55,NaN,1:29.229,53.0,18.0,215.364,Dutch,1981-09-23,Italian
23227,2005,Takuma Sato,BAR,Istanbul Park,14,Finished,9,0.0,58,+1:19.987,1:25.858,54.0,6.0,223.820,Japanese,1977-01-28,British
23234,2005,Narain Karthikeyan,Jordan,Istanbul Park,14,+3 Laps,14,0.0,55,NaN,1:29.286,42.0,19.0,215.227,Indian,1977-01-14,Irish
23236,2005,Tiago Monteiro,Jordan,Istanbul Park,14,+3 Laps,15,0.0,55,NaN,1:29.035,50.0,17.0,215.834,Portuguese,1976-07-24,Irish


In [26]:
df2 = df.copy()
df2['fastest_lap_time'] = df[df['fastest_lap_time'].isna()]

In [27]:
df2.groupby('circuit_name').agg({'fastest_lap_time': 'min'})

,fastest_lap_time
circuit_name,
A1-Ring,1970.0
AVUS,1959.0
Adelaide Street Circuit,1985.0
Ain Diab,1958.0
Aintree,1955.0
...,...
Valencia Street Circuit,2008.0
Watkins Glen,1961.0
Yas Marina Circuit,2010.0


#### Drivers results in top1 circuit

In [40]:
df_time = df[(df['position'] == 1) & (df['circuit_name'] == 'Autodromo Nazionale di Monza')]


KeyError: 'circuit_name'